经济学高校：16
经济学C刊数目：70
统计学高校：16
统计学C刊数目：4
管理学高校：10
管理学C刊数目：36
### 在最后，对每个学校的期刊来源进行数量统计，防止出现下载过程被中断、数据集未下载好但是被标记为完成状态的情况。

In [ ]:
注意事项： 最后统计的时候要格外注意，期刊名称是完全匹配，还是存在“包含”的情况

In [ ]:
#!/root/anaconda3/bin/python3
import pandas as pd
from time import sleep
import re
import threading
import random
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import os
from itertools import product

options = webdriver.ChromeOptions()
with open('经济学/succeed.txt','r',encoding='utf-8') as f:
    data = f.readlines()
succeed = [x.strip() for x in data] #记录完成的名单

mylock = threading.Lock() #给下载成功的名单加锁
def get_info(colleges,magazines):
    global options
    global succeed
    target_url = 'http://kns.cnki.net/kns/brief/result.aspx?dbprefix=CJFQ'
    #用笛卡尔积替代双重for循环，稳
    random.shuffle(colleges) #随机打乱
    random.shuffle(magazines)
    user_agent_list = [ \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60" \
        "Opera/8.0 (Windows NT 5.1; U; en)"\
        "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0" \
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER" \
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0" \
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)" \
    ]
    
    for college,magazine in product(colleges,magazines):
        failed_count = 0 #用于记录失败次数，暂时休息应对反爬
        nothing_flag = True #判断无检索结果还是成功导出
        the_item = college+'-'+magazine
        if the_item in succeed:
            print(the_item+' 已完成.')
            continue
        #print(college + ' ' + magazine)
        isExists = os.path.exists('经济学' + '/' + college) #创建学者文件夹
        if not isExists:
            os.makedirs('经济学' + '/' + college)
        UserAgent = 'user-agent=' + random.choice(user_agent_list)
        options.add_argument(UserAgent)
        index_path = 'D:\\codes\\GitProject\\工作区\\首经贸数据采集项目\\经济学'
        college_path = index_path + '\\' + college
        prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': college_path}  #下载路径
        options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(chrome_options=options)
        driver.implicitly_wait(2)
        try:  #访问
            driver.get(target_url)
        except:
            try:
                driver.refresh()
            except:
                print('超时错误')
                driver.quit()
                continue
        try:
            Select(driver.find_element_by_id('magazine_special1')).select_by_value('=') # 精确搜索
            mag = driver.find_element_by_id('magazine_value1')
            col = driver.find_element_by_id('au_1_value2')
            col.clear()
            col.send_keys(college)
            mag.clear()
            mag.send_keys(magazine) 
            mag.send_keys(Keys.ENTER)
            sleep(4)
            flag = True #用于定位iframe
            while True:  #判断是否有下一页
                while_flag = False
                for i in range(10): #10页导出一次
                    if flag:
                        driver.switch_to_frame('iframeResult')
                        to_clear = driver.find_element_by_class_name('zero')
                        to_clear.click()
                        driver.find_element_by_xpath('//*[@id="J_ORDER"]/tbody/tr[1]/td/table/tbody/tr/td[1]/span[3]/a').click()
                        sleep(4)
                        showPage = driver.find_element_by_xpath('//*[@id="id_grid_display_num"]/a[3]')
                        showPage.click() #每页显示50 
                    check_box = driver.find_element_by_id('selectCheckbox')
                    check_box.click()  # 全选
                    sleep(6)
                    #n_next = driver.find_element_by_class_name('TitleLeftCell')
                    try:
                        next_page = driver.find_element_by_link_text(u'下一页')
                    except:
                        sleep(3)
                        while_flag = True
                        #print('没有下一页')
                        break #没有下一页
                    next_page.click() #下一页
                    flag = False
                    sleep(5)
                #对查询结果为0的情况进行处理
                counts = driver.find_element_by_class_name('pagerTitleCell').text
                m = re.search('\d+',counts)
                if m:
                    if int(m.group(0)) == 0:
                        print(the_item+ ' 没有检索到成果信息.')
                        #driver.switch_to_window(driver.window_handles[0])
                        nothing_flag = False
                        sleep(2)
                        break
                export = driver.find_element_by_xpath('//*[@id="J_ORDER"]/tbody/tr[2]/td/table/tbody/tr/td[1]/div/a[2]')
                export.click()
                driver.switch_to_window(driver.window_handles[1]) #可能要重新定位windows
                sleep(3)
                zidingyi = driver.find_element_by_xpath('//*[@id="SaveTypeList"]/li[10]/span[1]/a')
                zidingyi.click()
                sleep(3)
                select_all = driver.find_element_by_xpath('//*[@id="selfDefine"]/table/tbody/tr[4]/td/input[1]')
                select_all.click()     
                download = driver.find_element_by_xpath('//*[@id="exportExcel"]')     #下载 excel
                download.click()
                sleep(5)
                #print('导出中..')
                driver.close()
                if while_flag: #跳出while循环，没有下一页
                    break
                driver.switch_to_window(driver.window_handles[0])
                driver.switch_to_frame('iframeResult')
                sleep(5)
                to_clear = driver.find_element_by_class_name('zero')
                to_clear.click()
            if nothing_flag:
                print(the_item + ' 导出成功。')
            if the_item not in succeed:
                succeed.append(the_item)
                succeed = list(set(succeed))
                if mylock.acquire(True):  #加锁
                    with open('经济学/succeed.txt','w',encoding='utf-8') as f:
                        for item in succeed:
                            f.write(item)
                            f.write('\n')
                    mylock.release()
                #print(the_item + ' 本地写入成功.')
            sleep(15)
            succeed  = list(set(succeed))
            if len(succeed) == 1050: #经济学的组合数为1050，其他组合再具体更改
                print('经济学（应用经济学）-南京大学C刊全部下载完毕。')
                break
            driver.quit() #关闭浏览器
        except:
            print(the_item+' 抓取失败')
            failed_count += 1
            if failed_count == 3: #如果连续3次失败，则跳出循环
                sleep(100)
            driver.quit()
        
def to_download(counts):
    data = pd.read_excel('高校名单.xlsx',sheet_name='经济学')
    colleges = data['高校名称']
    data = pd.read_excel('南京大学c刊.xlsx',sheet_name='经济学')
    magazines = data['期刊名称']
    for i in range(counts):
        print('第'+str(i+1)+'次循环')
        get_info(colleges,magazines)
        
if __name__ == '__main__':
    counts = 20
    threads = []
    for i in range(5):
        t = threading.Thread(target=to_download,args=(counts,))
        threads.append(t)
    for t in threads:
        t.start()
    for thread in threads:
        thread.join()


第1次循环
第1次循环
第1次循环
第1次循环
第1次循环
吉林大学-中国农村观察 导出成功。
浙江大学-金融论坛 导出成功。
江西财经大学-当代经济科学 导出成功。
吉林大学-贵州财经大学学报 抓取失败
江西财经大学-保险研究 导出成功。
浙江大学-世界经济文汇 导出成功。
吉林大学-中国农村经济 导出成功。
江西财经大学-财经论丛 导出成功。
吉林大学-劳动经济研究 导出成功。
浙江大学-经济理论与经济管理 导出成功。
江西财经大学-世界经济研究 导出成功。
吉林大学-经济理论与经济管理 导出成功。
